# Пробуем предсказать объем батарейки в телефоне

# Импорты

In [6]:
import pandas as pd # Библиотека Pandas для работы с табличными данными

from sklearn.model_selection import train_test_split #  функция разбиения на тренировочную и тестовую выборку
from sklearn.preprocessing import MinMaxScaler # Объект Нормализации от Scikit-learn
from sklearn.model_selection import cross_validate

In [2]:
from sklearn.neighbors import KNeighborsRegressor # Регрессия К-Ближайших соседей от scikit-learn

from sklearn.model_selection import ShuffleSplit # при кросс-валидации случайно перемешиваем данные

from sklearn.metrics import mean_squared_error as mse # метрика MSE от Scikit-learn
from sklearn.metrics import r2_score # коэффициент детерминации  от Scikit-learn

# Грузим данные

In [3]:
train = pd.read_csv('/content/mob_train.csv', delimiter = ',')
test = pd.read_csv('/content/mob_test.csv', delimiter = ',')

# Делим на тренировочную и тестовую

In [4]:
# не забываем удалить целевую переменную из признаков
X_train,y_train = train.drop(columns = ['battery_power']).values,train['battery_power'].values
X_test,y_test = test.drop(columns = ['battery_power']).values,test['battery_power'].values

# нормализуем
scaler  = MinMaxScaler()
scaler.fit(X_train) # для тренировочных сначала "обучаем"
X_train = scaler.transform(X_train) # потом преобразуем  transform
X_test  = scaler.transform(X_test)  # для тестовых - просто transform

# Обучаем модель

In [7]:
#@title Гиперпараметры к-Ближайших Соседей{ run: "auto" }
#@markdown ### количество соседей
k = 38 #@param {type:"slider", min:1, max:50, step:1}
#@markdown ### тип весов
weights = 'uniform' #@param [ 'uniform', 'distance']{type:"string"}


# создаем объект класса с указанием гиперпараметров
kNN = KNeighborsRegressor(n_neighbors=k,
                          weights=weights )

# обучаем на тренировочных данных
kNN.fit(X_train, y_train)

# предсказываем на тестовых данных
y_predict=kNN.predict(X_test)

# выбираем метрики которые будем оценивать
# в виде словаря
scoring = {'R2': 'r2',
           '-MSE': 'neg_mean_squared_error',
           '-MAE': 'neg_mean_absolute_error',
           'Max': 'max_error'}

# используем функцию кросс-валидации

scores = cross_validate(kNN, # какую модель тестируем
                        X_train, y_train, # на каких данных
                      scoring=scoring,  #на каких метриках
                      # как разбиваем данные при кросс-валидации
                      cv=ShuffleSplit(n_splits=5, random_state=42))


print('Результаты Кросс-валидации')
DF_cv_kNN = pd.DataFrame(scores)
display(DF_cv_kNN)
print('\n')
print(DF_cv_kNN.mean()[2:])

print('Ошибка на тестовых данных')
print('MSE: %.1f' % mse(y_test,y_predict))
print('RMSE: %.1f' % mse(y_test,y_predict,squared=False))
print('R2 : %.4f' %  r2_score(y_test,y_predict))

Результаты Кросс-валидации


,fit_time,score_time,test_R2,test_-MSE,test_-MAE,test_Max
0,0.001499,0.005379,0.025529,-193679.608192,-384.340977,-825.631579
1,0.001542,0.005410,0.043122,-204028.875237,-393.169549,-851.947368
2,0.001534,0.005470,0.027484,-184108.934893,-367.713158,-904.947368
3,0.001986,0.004915,0.001566,-208705.198778,-400.683271,-947.157895
4,0.001504,0.005310,0.030960,-184960.380392,-374.284586,-869.789474




test_R2           0.025732
test_-MSE   -195096.599498
test_-MAE      -384.038308
test_Max       -879.894737
dtype: float64
Ошибка на тестовых данных
MSE: 181406.6
RMSE: 425.9
R2 : 0.0223


In [16]:
df_scores = pd.read_csv('/content/scores.csv', delimiter = ',')

In [17]:
import datetime
now = datetime.datetime.now()
print(now)

2023-11-01 00:10:40.250378


In [25]:
df_scores

,date_n_time,MSE,RMSE,R2
0,2023-10-31 22:35:57.828090,206045.115761,453.921927,-0.110496
1,2023-11-01 00:10:40.250378,181406.608722,425.918547,0.022295


In [24]:
df_new_scores

,date_n_time,MSE,RMSE,R2
0,2023-11-01 00:10:40.250378,181406.608722,425.918547,0.022295


In [22]:
df_new_scores = pd.DataFrame([[now, mse(y_test,y_predict), mse(y_test,y_predict,squared=False), r2_score(y_test,y_predict)]], columns=['date_n_time', 'MSE', 'RMSE', 'R2'])

In [23]:
df_scores = df_scores.append(df_new_scores, ignore_index=True)

<ipython-input-23-8333813e2ac4>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append(df_new_scores, ignore_index=True)


In [26]:
df_scores.to_csv('scores.csv', index=False)

In [27]:
import pickle
with open("model.pkl", "wb") as f:
    pickle.dump(kNN, f)